# Creating a CSV agent
PythonREPL is a term used to describe the interactive shell or REPL (Read-Eval-Print Loop) in Python. It allows you to run Python code interactively, which is a useful tool for testing new ideas, exploring and experimenting with new tools and libraries, refactoring and debugging your code, and trying out examples


In [2]:
# Import relevant functionality
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent
from langchain_experimental.utilities import PythonREPL
from langchain_core.tools import Tool

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Create the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name="gpt4o",  # replace with your deployment name
    temperature=0.5,
    max_tokens=300,
    openai_api_version="2023-03-15-preview"
)

# Define the web_search tool
search = TavilySearchResults(max_results=2)
# Define the Python code execution tool
python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.",
    func=python_repl.run,
)

# Create the agent with memory and tools
tools = [search, repl_tool]
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Configuration for conversation thread
config = {"configurable": {"thread_id": "abc123"}}

In [4]:
# # testing the agent with a few messages

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="load the titanic.csv into a pandas dataframe. The file is located in the 'data' folder")]}, config
):   
        #print(chunk)
        try:
            if 'agent' in chunk:
                messages = chunk['agent']['messages'][0].content
                print('agent response')
                print(messages)
            elif 'tools' in chunk:
                messages = chunk['tools']['messages'][0].content
                print('tool response')
                print(messages)
            else:
                messages = []
        except KeyError:
            print("KeyError")
    

agent response



In [5]:
# ask follow up questions
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="how many rows are there in the titanic.csv?")]}, config
):
    #print(chunk)
        try:
            if 'agent' in chunk:
                messages = chunk['agent']['messages'][0].content
                print('agent response')
                print(messages)
            elif 'tools' in chunk:
                print(chunk)
                messages = chunk['tools']['messages'][0].content
                print('tool response')
                print(messages)
            else:
                messages = []
        except KeyError:
            print("KeyError")

BadRequestError: Error code: 400 - {'error': {'message': "An assistant message with 'tool_calls' must be followed by tool messages responding to each 'tool_call_id'. The following tool_call_ids did not have response messages: call_pSBFsmT4ZrbRljWkK3oUTF6h", 'type': 'invalid_request_error', 'param': 'messages.[2].role', 'code': None}}